# Segmenting and Clustering Neighborhoods in Toronto

## Introduction

In this notebook we'll convert addresses to geographic coordinates and use the Foursquare API to explore neighborhoods in Toronto. We'll try to find the most common venue categories in each neighborhood and then use *k*-means clustering to group neighborhoods and visualize emerging clusters.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1.  <a href="#item1">Download and Explore Dataset</a>

2.  <a href="#item2">Explore Neighborhoods in New York City</a>

3.  <a href="#item3">Analyze Each Neighborhood</a>

4.  <a href="#item4">Cluster Neighborhoods</a>

5.  <a href="#item5">Examine Clusters</a>  
    </font>
    </div>
    
First let's import and download all the libraries we'll need.

In [19]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library
import bs4 # webscraping library

from bs4 import BeautifulSoup
print('Libraries imported.')

Libraries imported.


In order to segment the city we'll need a data set that contains the neighborhood information. We can scrape such a table from Wikipedia using the `BeautifulSoup` library.

In [20]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data=requests.get(url).text

In [21]:
soup=BeautifulSoup(html_data,'html5lib')
table=soup.find('table')

In [22]:
table_contents=[]
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [23]:
n_data=pd.DataFrame(table_contents)
n_data

,Borough,Neighborhood,PostalCode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,"Regent Park, Harbourfront",M5A
3,North York,"Lawrence Manor, Lawrence Heights",M6A
4,Queen's Park,Ontario Provincial Government,M7A
...,...,...,...
98,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",M8X
99,Downtown Toronto,Church and Wellesley,M4Y
100,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,M7Y
101,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",M8Y


In [24]:
# import geocoder # import geocoder

# # initialize your variable to None
# lat_lng_coords = None
# postal_code='M3A'
# # loop until you get the coordinates
# while(lat_lng_coords is None):
#   ge = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#   lat_lng_coords = ge.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

In [31]:
url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'
geo_coords=pd.read_csv(url).rename(columns={'Postal Code':'PostalCode'})
geo_coords

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [32]:
n_data.merge(geo_coords,on="PostalCode")

,Borough,Neighborhood,PostalCode,Latitude,Longitude
0,North York,Parkwoods,M3A,43.753259,-79.329656
1,North York,Victoria Village,M4A,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,Queen's Park,Ontario Provincial Government,M7A,43.662301,-79.389494
...,...,...,...,...,...
98,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",M8X,43.653654,-79.506944
99,Downtown Toronto,Church and Wellesley,M4Y,43.665860,-79.383160
100,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,M7Y,43.662744,-79.321558
101,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",M8Y,43.636258,-79.498509
